# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [2]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


In [7]:
a=[]
for i in range (100):
    a.append(fizz_buzz_decode(i+1, fizz_buzz_encode(i+1)))

我们首先定义模型的输入与输出(训练数据)

选取100到2的10次方的数作为训练集,1到100的数作为测试集

In [10]:
import numpy as np
import torch

NUM_DIGITS = 10

# 定义一个函数，可以把一个十进制数转换为对应二进制数，二进制数每一位作为一个元素存在nparrry列表里
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])  
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])   #torch.LongTensor是64位整型，默认为32位浮点类型数据
trX.shape
#trX

C:\Users\dell\AppData\Local\Temp\ipykernel_24180\1261662670.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])


torch.Size([923, 10])

In [17]:
len(trX),trX.shape,trX[0]

(923, torch.Size([923, 10]), tensor([1., 0., 1., 0., 0., 1., 1., 0., 0., 0.]))

In [4]:
trY

tensor([0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2,
        0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2,
        0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1,

然后我们用PyTorch定义模型

In [18]:
# Define the model
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [19]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [20]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)  
        #torch.nn.CrossEntropyLoss()集成了对真实标签的onehot编码，以及对神经网络输出层的softmax处理求概率，
        #因此此处直接传入真实标签序号（无需onehot编码）以及神经网络输出层的值即可。
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.1755000352859497
Epoch: 1 Loss: 1.1523493528366089
Epoch: 2 Loss: 1.1467185020446777
Epoch: 3 Loss: 1.1445444822311401
Epoch: 4 Loss: 1.143436312675476
Epoch: 5 Loss: 1.1427481174468994
Epoch: 6 Loss: 1.142257571220398
Epoch: 7 Loss: 1.141873836517334
Epoch: 8 Loss: 1.141554832458496
Epoch: 9 Loss: 1.1412774324417114
Epoch: 10 Loss: 1.1410284042358398
Epoch: 11 Loss: 1.1407995223999023
Epoch: 12 Loss: 1.1405892372131348
Epoch: 13 Loss: 1.1403892040252686
Epoch: 14 Loss: 1.1402007341384888
Epoch: 15 Loss: 1.140023112297058
Epoch: 16 Loss: 1.1398499011993408
Epoch: 17 Loss: 1.1396853923797607
Epoch: 18 Loss: 1.1395244598388672
Epoch: 19 Loss: 1.1393702030181885
Epoch: 20 Loss: 1.1392194032669067
Epoch: 21 Loss: 1.1390750408172607
Epoch: 22 Loss: 1.138933777809143
Epoch: 23 Loss: 1.1387975215911865
Epoch: 24 Loss: 1.1386632919311523
Epoch: 25 Loss: 1.138533115386963
Epoch: 26 Loss: 1.1384037733078003
Epoch: 27 Loss: 1.1382794380187988
Epoch: 28 Loss: 1.1381580829620361
Ep

Epoch: 716 Loss: 0.8596638441085815
Epoch: 717 Loss: 0.8590821623802185
Epoch: 718 Loss: 0.8578283786773682
Epoch: 719 Loss: 0.857660710811615
Epoch: 720 Loss: 0.8565470576286316
Epoch: 721 Loss: 0.8551192879676819
Epoch: 722 Loss: 0.8545296788215637
Epoch: 723 Loss: 0.8536115288734436
Epoch: 724 Loss: 0.8531513214111328
Epoch: 725 Loss: 0.8520618081092834
Epoch: 726 Loss: 0.8514078259468079
Epoch: 727 Loss: 0.8498461842536926
Epoch: 728 Loss: 0.8487592339515686
Epoch: 729 Loss: 0.8481102585792542
Epoch: 730 Loss: 0.8479304313659668
Epoch: 731 Loss: 0.8473015427589417
Epoch: 732 Loss: 0.8462771773338318
Epoch: 733 Loss: 0.844771683216095
Epoch: 734 Loss: 0.8443911671638489
Epoch: 735 Loss: 0.8431770205497742
Epoch: 736 Loss: 0.8430683612823486
Epoch: 737 Loss: 0.841506838798523
Epoch: 738 Loss: 0.8402745723724365
Epoch: 739 Loss: 0.8397330641746521
Epoch: 740 Loss: 0.8388100266456604
Epoch: 741 Loss: 0.8383536338806152
Epoch: 742 Loss: 0.836919367313385
Epoch: 743 Loss: 0.8363997936248

Epoch: 973 Loss: 0.5725299715995789
Epoch: 974 Loss: 0.5724585652351379
Epoch: 975 Loss: 0.5707054734230042
Epoch: 976 Loss: 0.5707384347915649
Epoch: 977 Loss: 0.5684239864349365
Epoch: 978 Loss: 0.5688055157661438
Epoch: 979 Loss: 0.566804051399231
Epoch: 980 Loss: 0.565423846244812
Epoch: 981 Loss: 0.5651766657829285
Epoch: 982 Loss: 0.5629198551177979
Epoch: 983 Loss: 0.5635009407997131
Epoch: 984 Loss: 0.5609162449836731
Epoch: 985 Loss: 0.5603814721107483
Epoch: 986 Loss: 0.5591437220573425
Epoch: 987 Loss: 0.5584094524383545
Epoch: 988 Loss: 0.5575730800628662
Epoch: 989 Loss: 0.556292712688446
Epoch: 990 Loss: 0.5561778545379639
Epoch: 991 Loss: 0.5536942481994629
Epoch: 992 Loss: 0.5537667870521545
Epoch: 993 Loss: 0.5518586039543152
Epoch: 994 Loss: 0.551224410533905
Epoch: 995 Loss: 0.5507449507713318
Epoch: 996 Loss: 0.5487471222877502
Epoch: 997 Loss: 0.5481716990470886
Epoch: 998 Loss: 0.5470220446586609
Epoch: 999 Loss: 0.5454778671264648
Epoch: 1000 Loss: 0.545701801776

Epoch: 1208 Loss: 0.3905300199985504
Epoch: 1209 Loss: 0.39024657011032104
Epoch: 1210 Loss: 0.3892630636692047
Epoch: 1211 Loss: 0.3886198103427887
Epoch: 1212 Loss: 0.3894045352935791
Epoch: 1213 Loss: 0.3873920440673828
Epoch: 1214 Loss: 0.3866465091705322
Epoch: 1215 Loss: 0.3866738975048065
Epoch: 1216 Loss: 0.3856900930404663
Epoch: 1217 Loss: 0.3864066004753113
Epoch: 1218 Loss: 0.3855040669441223
Epoch: 1219 Loss: 0.3842275142669678
Epoch: 1220 Loss: 0.3834535777568817
Epoch: 1221 Loss: 0.3830540180206299
Epoch: 1222 Loss: 0.3823121190071106
Epoch: 1223 Loss: 0.381438285112381
Epoch: 1224 Loss: 0.38115981221199036
Epoch: 1225 Loss: 0.3804309368133545
Epoch: 1226 Loss: 0.3799882233142853
Epoch: 1227 Loss: 0.37955549359321594
Epoch: 1228 Loss: 0.379052996635437
Epoch: 1229 Loss: 0.3798692524433136
Epoch: 1230 Loss: 0.3781938850879669
Epoch: 1231 Loss: 0.377398282289505
Epoch: 1232 Loss: 0.3782740533351898
Epoch: 1233 Loss: 0.3773188591003418
Epoch: 1234 Loss: 0.377034455537796
Ep

Epoch: 1447 Loss: 0.27754032611846924
Epoch: 1448 Loss: 0.2768692374229431
Epoch: 1449 Loss: 0.27654191851615906
Epoch: 1450 Loss: 0.276064932346344
Epoch: 1451 Loss: 0.2757045030593872
Epoch: 1452 Loss: 0.27622488141059875
Epoch: 1453 Loss: 0.27568864822387695
Epoch: 1454 Loss: 0.2746981382369995
Epoch: 1455 Loss: 0.27498379349708557
Epoch: 1456 Loss: 0.2739349603652954
Epoch: 1457 Loss: 0.2738453447818756
Epoch: 1458 Loss: 0.2733120024204254
Epoch: 1459 Loss: 0.27278387546539307
Epoch: 1460 Loss: 0.27339044213294983
Epoch: 1461 Loss: 0.27198538184165955
Epoch: 1462 Loss: 0.2717237174510956
Epoch: 1463 Loss: 0.2716265618801117
Epoch: 1464 Loss: 0.27090984582901
Epoch: 1465 Loss: 0.27142244577407837
Epoch: 1466 Loss: 0.2701912522315979
Epoch: 1467 Loss: 0.26980486512184143
Epoch: 1468 Loss: 0.2699873149394989
Epoch: 1469 Loss: 0.26977062225341797
Epoch: 1470 Loss: 0.2695184350013733
Epoch: 1471 Loss: 0.26851919293403625
Epoch: 1472 Loss: 0.26822078227996826
Epoch: 1473 Loss: 0.26765877

...

Epoch: 9023 Loss: 0.009100557304918766
Epoch: 9024 Loss: 0.009098559617996216
Epoch: 9025 Loss: 0.009097890928387642
Epoch: 9026 Loss: 0.00909594539552927
Epoch: 9027 Loss: 0.009092926979064941
Epoch: 9028 Loss: 0.009092271327972412
Epoch: 9029 Loss: 0.009089086204767227
Epoch: 9030 Loss: 0.009087709710001945
Epoch: 9031 Loss: 0.009084888733923435
Epoch: 9032 Loss: 0.00908263586461544
Epoch: 9033 Loss: 0.009083706885576248
Epoch: 9034 Loss: 0.009082156233489513
Epoch: 9035 Loss: 0.009078377857804298
Epoch: 9036 Loss: 0.009076646529138088
Epoch: 9037 Loss: 0.009072673507034779
Epoch: 9038 Loss: 0.00907517597079277
Epoch: 9039 Loss: 0.009071306325495243
Epoch: 9040 Loss: 0.009070423431694508
Epoch: 9041 Loss: 0.009068232029676437
Epoch: 9042 Loss: 0.009067163802683353
Epoch: 9043 Loss: 0.009063723497092724
Epoch: 9044 Loss: 0.009062333032488823
Epoch: 9045 Loss: 0.009060593321919441
Epoch: 9046 Loss: 0.009059968404471874
Epoch: 9047 Loss: 0.009056913666427135
Epoch: 9048 Loss: 0.00905515

Epoch: 9243 Loss: 0.00871984101831913
Epoch: 9244 Loss: 0.008716662414371967
Epoch: 9245 Loss: 0.008716716431081295
Epoch: 9246 Loss: 0.008715400472283363
Epoch: 9247 Loss: 0.008712133392691612
Epoch: 9248 Loss: 0.008711711503565311
Epoch: 9249 Loss: 0.008710267953574657
Epoch: 9250 Loss: 0.008708417415618896
Epoch: 9251 Loss: 0.008706023916602135
Epoch: 9252 Loss: 0.008704441599547863
Epoch: 9253 Loss: 0.008701859973371029
Epoch: 9254 Loss: 0.008702346123754978
Epoch: 9255 Loss: 0.008701960556209087
Epoch: 9256 Loss: 0.008698174729943275
Epoch: 9257 Loss: 0.008697746321558952
Epoch: 9258 Loss: 0.00869471300393343
Epoch: 9259 Loss: 0.008691876195371151
Epoch: 9260 Loss: 0.00869070552289486
Epoch: 9261 Loss: 0.00868936162441969
Epoch: 9262 Loss: 0.008688768371939659
Epoch: 9263 Loss: 0.008685918524861336
Epoch: 9264 Loss: 0.008685190230607986
Epoch: 9265 Loss: 0.008682518266141415
Epoch: 9266 Loss: 0.008682811632752419
Epoch: 9267 Loss: 0.008681503124535084
Epoch: 9268 Loss: 0.008678263

Epoch: 9472 Loss: 0.00835282914340496
Epoch: 9473 Loss: 0.00835311971604824
Epoch: 9474 Loss: 0.008349591866135597
Epoch: 9475 Loss: 0.008347484283149242
Epoch: 9476 Loss: 0.00834596436470747
Epoch: 9477 Loss: 0.008344866335391998
Epoch: 9478 Loss: 0.008343381807208061
Epoch: 9479 Loss: 0.008341182954609394
Epoch: 9480 Loss: 0.0083396527916193
Epoch: 9481 Loss: 0.008339089341461658
Epoch: 9482 Loss: 0.008336551487445831
Epoch: 9483 Loss: 0.008336031809449196
Epoch: 9484 Loss: 0.008336756378412247
Epoch: 9485 Loss: 0.008332014083862305
Epoch: 9486 Loss: 0.008332205936312675
Epoch: 9487 Loss: 0.008329283446073532
Epoch: 9488 Loss: 0.008328509517014027
Epoch: 9489 Loss: 0.008326182141900063
Epoch: 9490 Loss: 0.00832381285727024
Epoch: 9491 Loss: 0.008323704823851585
Epoch: 9492 Loss: 0.008320248685777187
Epoch: 9493 Loss: 0.00832049548625946
Epoch: 9494 Loss: 0.008321286179125309
Epoch: 9495 Loss: 0.008318078704178333
Epoch: 9496 Loss: 0.008314809761941433
Epoch: 9497 Loss: 0.008312897756

Epoch: 9696 Loss: 0.008023802191019058
Epoch: 9697 Loss: 0.008020234294235706
Epoch: 9698 Loss: 0.008019068278372288
Epoch: 9699 Loss: 0.00801615510135889
Epoch: 9700 Loss: 0.008017376065254211
Epoch: 9701 Loss: 0.008015263825654984
Epoch: 9702 Loss: 0.008014575578272343
Epoch: 9703 Loss: 0.008012527599930763
Epoch: 9704 Loss: 0.008011060766875744
Epoch: 9705 Loss: 0.00800999440252781
Epoch: 9706 Loss: 0.008008936420083046
Epoch: 9707 Loss: 0.008007816039025784
Epoch: 9708 Loss: 0.008005008101463318
Epoch: 9709 Loss: 0.00800387654453516
Epoch: 9710 Loss: 0.008003146387636662
Epoch: 9711 Loss: 0.00800394918769598
Epoch: 9712 Loss: 0.00799943320453167
Epoch: 9713 Loss: 0.00799701176583767
Epoch: 9714 Loss: 0.007997491396963596
Epoch: 9715 Loss: 0.007995646446943283
Epoch: 9716 Loss: 0.007994724437594414
Epoch: 9717 Loss: 0.007993181236088276
Epoch: 9718 Loss: 0.007990850135684013
Epoch: 9719 Loss: 0.007989685982465744
Epoch: 9720 Loss: 0.00798824243247509
Epoch: 9721 Loss: 0.007987407967

Epoch: 9907 Loss: 0.007735022343695164
Epoch: 9908 Loss: 0.007732315920293331
Epoch: 9909 Loss: 0.007732408586889505
Epoch: 9910 Loss: 0.007731896359473467
Epoch: 9911 Loss: 0.0077292839996516705
Epoch: 9912 Loss: 0.007727726362645626
Epoch: 9913 Loss: 0.007725482806563377
Epoch: 9914 Loss: 0.007726293057203293
Epoch: 9915 Loss: 0.007723990362137556
Epoch: 9916 Loss: 0.007722488138824701
Epoch: 9917 Loss: 0.00772323552519083
Epoch: 9918 Loss: 0.007719853427261114
Epoch: 9919 Loss: 0.007716689724475145
Epoch: 9920 Loss: 0.007716765161603689
Epoch: 9921 Loss: 0.0077170440927147865
Epoch: 9922 Loss: 0.007715580053627491
Epoch: 9923 Loss: 0.007715020794421434
Epoch: 9924 Loss: 0.007712380960583687
Epoch: 9925 Loss: 0.007710346952080727
Epoch: 9926 Loss: 0.007709023542702198
Epoch: 9927 Loss: 0.007706128526479006
Epoch: 9928 Loss: 0.007706046104431152
Epoch: 9929 Loss: 0.007706144824624062
Epoch: 9930 Loss: 0.007703952491283417
Epoch: 9931 Loss: 0.0077019548043608665
Epoch: 9932 Loss: 0.007

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [21]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', '10', '11', '12', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', 'buzz', 'buzz', 'buzz', '37', 'buzz', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', '51', '52', '53', 'fizz', 'buzz', '56', '57', '58', '59', '60', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', '90', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', 'fizz', 'fizz', 'buzz']


In [22]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

90


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True])